In [1]:
# Імпорт бібілотек
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score

# Завантаження даних
train_data = pd.read_csv('E:/Machine Learning/final_proj_data.csv')
test_data = pd.read_csv('E:/Machine Learning/final_proj_test.csv')

# Видалення ознак, що мають більше 35% пропусків
def remove_high_missing_columns(df, threshold=0.35):
    missing_percentage = df.isnull().mean()
    return df.loc[:, missing_percentage < threshold]

train_data = remove_high_missing_columns(train_data)
test_data = remove_high_missing_columns(test_data)

# Відокремлення функцій і цільової змінної
X = train_data.drop('y', axis=1)
y = train_data['y']

# Розділення даних на тренувальний та валідаційний набори
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Визначення категорійних та числових ознак
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
numeric_features = [col for col in X.columns if col not in categorical_features + ['y']]

# Підготовка конвеєра для передобробки з вибором найкращих ознак
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
            ('scaler', StandardScaler()),
            ('selectkbest', SelectKBest(f_classif, k=30))  # Вибір 30 найкращих ознак
        ]), numeric_features),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
            ('selectkbest', SelectKBest(f_classif, k=100))  # Вибір 100 найкращих категорійних ознак
        ]), categorical_features)
    ])

# Визначення моделей
knn_model = KNeighborsClassifier(n_neighbors=20)

svm_model = SVC(probability=True, random_state=42)

xgb_model = XGBClassifier(
    n_estimators=50,
    learning_rate=0.1,
    max_depth=5,
    random_state=42
)

rf_model = RandomForestClassifier(
    n_estimators=30,
    max_depth=10,
    random_state=42
)

gb_model = GradientBoostingClassifier(
    n_estimators=30,
    learning_rate=0.1,
    max_depth=5,
    random_state=42
)

bagging_model = BaggingClassifier(
    estimator=DecisionTreeClassifier(random_state=42),
    n_estimators=50,
    random_state=42
)

# Створення ансамблю моделей
ensemble_model = VotingClassifier(
    estimators=[
        ('knn', knn_model),
        ('svm', svm_model),
        ('rf', rf_model),
        ('gb', gb_model),
        ('bagging', bagging_model),
        ('xgb', xgb_model)
    ],
    voting='soft'
)

# Створення пайплайну
pipeline = make_pipeline(
    preprocessor,
    SMOTE(random_state=42),
    ensemble_model
)

# Оцінка моделі за допомогою крос-валідації
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='balanced_accuracy')
print(f"Cross-validation Balanced Accuracy scores: {cv_scores}")
print(f"Mean CV Balanced Accuracy: {np.mean(cv_scores):.4f}")

# Навчання моделі на всьому тренувальному наборі
pipeline.fit(X_train, y_train)

# Оцінка на валідаційному наборі
y_pred = pipeline.predict(X_val)
y_pred_proba = pipeline.predict_proba(X_val)[:, 1]

print(f"Validation Accuracy: {accuracy_score(y_val, y_pred):.4f}")
print(f"Validation Balanced Accuracy: {balanced_accuracy_score(y_val, y_pred):.4f}")
print(f"Validation ROC AUC: {roc_auc_score(y_val, y_pred_proba):.4f}")

# Навчання фінальної моделі на всьому наборі даних
pipeline.fit(X, y)

# Прогнозування для тестового набору
predictions = pipeline.predict(test_data)

# Створення файлу для подання
submission = pd.DataFrame({'index': test_data.index, 'y': predictions})
submission.to_csv('E:/Machine Learning/submission_29.csv', index=False)

Cross-validation Balanced Accuracy scores: [0.83733974 0.83267519 0.8549252  0.83183246 0.87431162]
Mean CV Balanced Accuracy: 0.8462
Validation Accuracy: 0.8760
Validation Balanced Accuracy: 0.8652
Validation ROC AUC: 0.9459
